In [2]:
!uv pip install git+https://github.com/maraxen/py2Dmol

Using Python 3.12.12 environment at: /usr
Resolved 23 packages in 536ms
Prepared 1 package in 2.05s
Uninstalled 1 package in 0.61ms
Installed 1 package in 2ms
 - py2dmol==1.1.3
 + py2dmol==1.1.0 (from git+https://github.com/maraxen/py2Dmol@3ff9cdc6187545d5114c74621fdc7502c101eee7)


In [3]:
import tempfile

import requests

from py2Dmol import View

view = View()


# py2Dmol View: Example and API overview

This notebook demonstrates how to use the View class from py2Dmol. Key capabilities:

- Instantiate a viewer with size and color scheme: View(size=(width,height), color='rainbow'|'monochrome'|...).
- Add frames of coordinates programmatically with add(coords, plddts=None, chains=None, atom_types=None, new_traj=False).
- Start new trajectories using new_traj=True (useful to separate unrelated models).
- Load structures from files using add_pdb(filepath, chains=None) or from_pdb(filepath, new_traj=True).
- Clear the viewer with clear().
- The viewer will align subsequently added frames to the first frame automatically (Kabsch).

Below are compact examples covering these patterns.

In [4]:
# Example: create a viewer with custom size and color, then add programmatic frames
import numpy as np

from py2Dmol import View

# Create viewer with a custom size and color scheme
view = View(size=(700, 350), color="rainbow")

In [5]:
# Make a small example molecule (3 points) and add as first frame (start a new trajectory)
coords1 = np.array([[0.0, 0.0, 0.0],
                    [1.0, 0.0, 0.0],
                    [0.0, 1.0, 0.0]])
plddts1 = np.array([90.0, 80.0, 85.0])
chains1 = ["A", "A", "A"]
atom_types1 = ["P", "P", "P"]

view.add(coords1, plddts1, chains1, atom_types1, new_traj=True)

# Add a second frame: small translation — will be aligned to the first frame
coords2 = coords1 + np.array([0.2, 0.1, 0.0])
view.add(coords2, plddts1, chains1, atom_types1)

# Start a completely separate trajectory (new_traj=True)
coords3 = coords1 * 1.5
view.add(coords3, plddts1, chains1, atom_types1, new_traj=True)

# You can clear the viewer when needed
# view.clear()

In [6]:
# Load a PDB from the RCSB and display it. This uses a temporary file.
view.clear()  # Clear previous content
url = "https://files.rcsb.org/download/1BNA.pdb"
pdb_data = requests.get(url, timeout=10).text

with tempfile.NamedTemporaryFile(suffix=".pdb") as temp_pdb:
    temp_pdb.write(pdb_data.encode())
    temp_pdb.flush()
    temp_path = temp_pdb.name
    # Two common options:
    # 1) add_pdb() will append the parsed coordinates as frames (multi-models become frames).
    view.add_pdb(temp_path)
    # 2) from_pdb() is a convenience wrapper that starts a new trajectory by default:
    # view.from_pdb(temp_path, chains=None, new_traj=True)


## Viewing Trajectories

py2Dmol also allows you to visualize trajectories or multi-model structures.

In [7]:
# Load a PDB from the RCSB and display it. This uses a temporary file.
view.clear()  # Clear previous content
url = "https://files.rcsb.org/download/2KPO.pdb"
pdb_data = requests.get(url, timeout=10).text

with tempfile.NamedTemporaryFile(suffix=".pdb") as temp_pdb:
    temp_pdb.write(pdb_data.encode())
    temp_pdb.flush()
    temp_path = temp_pdb.name
    # Two common options:
    # 1) add_pdb() will append the parsed coordinates as frames (multi-models become frames).
    view.add_pdb(temp_path)
    # 2) from_pdb() is a convenience wrapper that starts a new trajectory by default:
    # view.from_pdb(temp_path, chains=None, new_traj=True)


Notes and tips:

- If you want to load only specific chains from a PDB/CIF file, pass chains=['A','B'] to add_pdb/from_pdb.
- pLDDT values are read from the B-factor field when available; you can also provide your own plddts array.
- Atom types control iconography in the viewer (e.g., 'P' for protein CA, 'R'/'D' for RNA/DNA, 'L' for ligand).
- The viewer uses an iframe-based front end and works in Jupyter and Colab (it uses a different messaging path in Colab).
- Use new_traj=True to separate logically distinct models so they appear as independent trajectories.

For advanced usage you can build coordinate arrays from MD frames or other sources and stream them into the viewer with add(...) repeatedly.